# RSNA evaluation

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from hydra import compose, initialize
sys.path.append("/vol/biomedic3/mb121/causal-contrastive")
from classification.classification_module import ClassificationModule
from data_handling.xray import RSNAPneumoniaDataModule
from evaluation.helper_functions import run_inference
os.chdir("/vol/biomedic3/mb121/causal-contrastive/evaluation")

In [ ]:
# Mapping from human readable run name to Weights&Biases run_id. 

# Human readable name should be in format:
# for finetuning:
# {simclr/simclrcf/simclrcfaug}-{train_prop}-{seed}
# for linear probing
# {simclr/simclrcf/simclrcfaug}head-{train_prop}-{seed}

model_dict_normal = {
    "simclrhead-0.1-22": "zcwca4kp",
    "simclrhead-1.0-22": "2wdyk6sc",
    "simclrhead-0.1-11": "rh8939mj",
    "simclrhead-1.0-11": "c8eax3ze",
    "simclrhead-0.1-33": "3liv3pgo",
    "simclrhead-1.0-33": "4m7p3834",
    "supervised-0.1-11": "lnofy8px",
    "supervised-0.1-22": "ck3nhurh",
    "supervised-0.1-33": "fq85b7yv",
    "supervised-1.0-11": "iasr1ngi",
    "supervised-1.0-22": "aq28vr4n",
    "supervised-1.0-33": "jzecmpej",
    "simclrcfaug-1.0-33": "johrigco",
    "simclrcfaug-0.1-33": "mwj1n15o",
    "simclrcfaug-1.0-22": "urupcr8p",
    "simclrcfaug-0.1-22": "zhqlwbiv",
    "simclrcfaug-1.0-11": "u065scjn",
    "simclrcfaug-0.1-11": "10ik4o6r",
    "simclrcf-0.1-11": "dmhnp0bi",
    "simclrcf-1.0-11": "exuhveyd",
    "simclrcf-0.1-22": "79sei22s",
    "simclrcf-1.0-22": "pslj9cw7",
    "simclrcf-0.1-33": "q3poikhz",
    "simclrcf-1.0-33": "fjb1jc33",
    "simclrcf-0.25-11": "drpdrih7",
    "simclrcf-0.25-22": "ejxf43d7",
    "simclrcf-0.25-33": "nvgwji8u",
    "simclr-0.25-11": "4o9d80gc",
    "simclr-0.25-22": "o3vvj758",
    "simclr-0.25-33": "r3iwi9qf",
    "simclrcfaug-0.25-11": "ia70qkb8",
    "simclrcfaug-0.25-22": "u4npk2ti",
    "simclrcfaug-0.25-33": "wjx8atvt",
    "simclr-0.1-11": "ie25zeoi",
    "simclr-0.1-22": "wkgvu1ij",
    "simclr-0.1-33": "r6ga87wc",
    "simclr-1.0-11": "tgdoaxmj",
    "simclr-1.0-22": "gxhens59",
    "simclr-1.0-33": "hsxljb6i",
    "simclrcfhead-0.1-11": "8f7bsf6b",
    "simclrcfhead-0.25-11": "w9o8807c",
    "simclrcfhead-1.0-11": "lv68umt5",
    "simclrcfhead-0.1-22": "4yittl5i",
    "simclrcfhead-0.25-22": "q8fg06rd",
    "simclrcfhead-1.0-22": "4pos6qck",
    "simclrcfhead-0.1-33": "btyurdej",
    "simclrcfaughead-0.1-11": "l7j8qxc8",
    "simclrcfaughead-0.25-11": "hj12lhso",
    "simclrcfaughead-1.0-11": "00szhgxn",
    "simclrcfaughead-0.1-22": "01vil6ce",
    "simclrcfaughead-0.25-22": "4fx9i1au",
    "simclrcfaughead-1.0-22": "dc8a5vzx",
    "simclrcfaughead-0.1-33": "nov89a96",
    "simclrcfaughead-0.25-33": "ll31rldz",
    "simclrcfaughead-1.0-33": "fvj2uqpf",
    "simclrcfhead-1.0-33": "vp68n1ii",
    "simclrcfhead-0.25-33": "8kauu1kh",
    "supervised-0.25-11": 'vgbe3kaw',
    "supervised-0.25-22": 'tb0gu42b',
    "supervised-0.25-33": 'uc7ew0bk'
}

In [ ]:
with initialize(version_base=None, config_path="../configs"):
    cfg = compose(
        config_name="config.yaml", overrides=["experiment=base_rsna", "data.cache=True"]
    )
    data_module = RSNAPneumoniaDataModule(config=cfg)

test_dataloader = data_module.test_dataloader()

In [ ]:
df = pd.read_csv(f"../outputs/classification_rsna_results.csv")
for run_name, run_id in model_dict_normal.items():
    already_in_df = run_name in df.run_name.values
    if run_id != "" and not already_in_df:
        print(run_name)
        model_to_evaluate = f"../outputs/run_{run_id}/best.ckpt"
        classification_model = ClassificationModule.load_from_checkpoint(
            model_to_evaluate, map_location="cuda:0", strict=False
        ).model.eval()
        classification_model.cuda()
        inference_results = run_inference(test_dataloader, classification_model)
        print(f"\nEvaluating RSNA")
        res = {}
        res["N_test"] = [inference_results["targets"].shape[0]]
        res["Scanner"] = ["RSNA"]
        res["run_name"] = run_name
        res["ROC"] = [
            roc_auc_score(
                inference_results["targets"], inference_results["confs"][:, 1]
            )
        ]
        print(res)
        df = pd.concat([df, pd.DataFrame(res, index=[0])], ignore_index=True)
        df.to_csv(f"../outputs/classification_rsna_results.csv", index=False)